In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Projects/Perturbed-Equilibrium/JPEC.worktrees/vacuum_julia`


In [2]:
function assemble_vacuum_matrix!(    
    vacmat, vacmti, vacmatu, vacmtiu,
    grwp, grpw, grwpw, grww,
    mth, jmax1, ln, lx, factpi
)
    """
    assemble_vacuum_matrix!(
        vacmat, vacmti, vacmatu, vacmtiu,
        grwp, grpw, grwpw, grww,
        mth, jmax1, ln, lx, factpi
    )

    Assemble the full vacuum response matrix from Green’s function sub-blocks.

    This routine constructs the four quadrants of the vacuum matrix that couple
    plasma and wall Fourier modes. The block structure is:

        ┌                 ┐
        │  PP   PW        │
        │                 │
        │  WP   WW        │
        └                 ┘

    where:
    - PP = Plasma–Plasma block
    - PW = Plasma–Wall block
    - WP = Wall–Plasma block
    - WW = Wall–Wall block

    Each sub-block is filled from the corresponding Green’s function kernel arrays.

    Arguments
    ---------
    - `vacmat::Array{Float64,2}` : Main vacuum response matrix (modified in place).
    - `vacmti::Array{Float64,2}` : Work array for inverse/iterative solver (zeroed here).
    - `vacmatu::Array{Float64,2}` : Work array for upper triangular factor (zeroed here).
    - `vacmtiu::Array{Float64,2}` : Work array for inverse of upper factor (zeroed here).
    - `grwp::Array{Float64,2}` : Green’s function values for Plasma–Plasma coupling.
    - `grpw::Array{Float64,2}` : Green’s function values for Plasma–Wall coupling.
    - `grwpw::Array{Float64,2}` : Green’s function values for Wall–Plasma coupling.
    - `grww::Array{Float64,2}` : Green’s function values for Wall–Wall coupling.
    - `mth::Int` : Number of poloidal modes.
    - `jmax1::Int` : Number of radial Fourier indices (per mode).
    - `ln::Int` : Toroidal mode index (reserved, not used here).
    - `lx::Int` : Radial surface index (reserved, not used here).
    - `factpi::Float64` : Normalization factor (π-related).

    Notes
    -----
    - The matrices `vacmti`, `vacmatu`, and `vacmtiu` are reset to zero here
    but not assembled; they are prepared for later linear algebra operations.
    - Indexing follows the (m, l) block structure, flattened to (row, col).
    - The resulting `vacmat` has dimensions `(2*mth*jmax1, 2*mth*jmax1)`.
    """
    
    # Dimensions
    nrows = 2 * mth * jmax1
    ncols = 2 * mth * jmax1

    # Reset all vacuum matrices
    vacmat  .= 0.0
    vacmti  .= 0.0
    vacmatu .= 0.0
    vacmtiu .= 0.0

    # ----------------------------------------------------------
    # Assemble Plasma–Plasma block into vacmat (upper-left)
    # ----------------------------------------------------------
    for m in 1:mth
        for l in 1:jmax1
            row = (m - 1) * jmax1 + l
            for mp in 1:mth
                for lp in 1:jmax1
                    col = (mp - 1) * jmax1 + lp
                    vacmat[row, col] += grwp[row, col] * factpi
                end
            end
        end
    end

    # ----------------------------------------------------------
    # Assemble Plasma–Wall block into vacmat (upper-right)
    # ----------------------------------------------------------
    offset_col = mth * jmax1
    for m in 1:mth
        for l in 1:jmax1
            row = (m - 1) * jmax1 + l
            for mp in 1:mth
                for lp in 1:jmax1
                    col = offset_col + (mp - 1) * jmax1 + lp
                    vacmat[row, col] += grpw[row, col - offset_col] * factpi
                end
            end
        end
    end

    # ----------------------------------------------------------
    # Assemble Wall–Plasma block into vacmat (lower-left)
    # ----------------------------------------------------------
    offset_row = mth * jmax1
    for m in 1:mth
        for l in 1:jmax1
            row = offset_row + (m - 1) * jmax1 + l
            for mp in 1:mth
                for lp in 1:jmax1
                    col = (mp - 1) * jmax1 + lp
                    vacmat[row, col] += grwpw[row - offset_row, col] * factpi
                end
            end
        end
    end

    # ----------------------------------------------------------
    # Assemble Wall–Wall block into vacmat (lower-right)
    # ----------------------------------------------------------
    for m in 1:mth
        for l in 1:jmax1
            row = offset_row + (m - 1) * jmax1 + l
            for mp in 1:mth
                for lp in 1:jmax1
                    col = offset_col + (mp - 1) * jmax1 + lp
                    vacmat[row, col] += grww[row - offset_row, col - offset_col] * factpi
                end
            end
        end
    end

    return nothing
end

assemble_vacuum_matrix! (generic function with 1 method)

In [3]:
function solve_vacuum!(
    vacmat, vacmti, vacmatu, vacmtiu,
    rhs, chi,
    use_updown::Bool=false, use_symmetry::Bool=false
)
    """
    Solve the vacuum matrix equation for the Fourier coefficients.

    Parameters
    ----------
    vacmat, vacmti, vacmatu, vacmtiu : Arrays
        Vacuum matrices (preassembled).
    rhs : Vector
        Right-hand side vector (plasma boundary conditions).
    chi : Vector (preallocated)
        Output solution vector (vacuum potential coefficients).
    use_updown : Bool
        If true, use the up–down symmetric version.
    use_symmetry : Bool
        If true, use the toroidal symmetry version.

    Returns
    -------
    chi : updated in place
    """

    # Select which matrix to use based on flags
    mat = vacmat
    if use_updown && use_symmetry
        mat = vacmtiu
    elseif use_updown
        mat = vacmatu
    elseif use_symmetry
        mat = vacmti
    end

    # Solve system: mat * chi = rhs
    try
        chi .= mat \ rhs
    catch err
        @warn "Vacuum solve failed, using fallback (zeros)" exception=(err, catch_backtrace())
        chi .= 0.0
    end

    return nothing
end

solve_vacuum! (generic function with 3 methods)

In [4]:
function felang!(grwp::Array{Float64,2}, grri::Array{Float64,2},
                 coeffs::Array{Float64,2},
                 offset_i::Int, offset_j::Int)
    """
    Perform Fourier-Legendre angular expansion (placeholder).

    Parameters
    ----------
    grwp : 2D Array
        Green’s function workspace (observer × source grid).
    grri : 2D Array
        Green’s function real part (or additional workspace).
    coeffs : 2D Array
        Output array for Fourier coefficients.
    offset_i : Int
        Row offset for the slice of grwp to use.
    offset_j : Int
        Column offset for the slice of grri to use.

    Returns
    -------
    coeffs : updated in place
    """

    nrows, ncols = size(grwp)
    jmax = min(size(coeffs,2), ncols - offset_j)

    # Loop over Fourier mode index (like l1 in Fortran)
    for l in 1:jmax
        # Fortran: sum over theta index (like i in Fortran loop)
        s = 0.0
        for i in 1:nrows
            ii = offset_i + i
            jj = offset_j + l
            # Protect indices in case offsets push them out of bounds
            if 1 <= ii <= size(grwp,1) && 1 <= jj <= size(grri,2)
                s += grwp[ii,1] * grri[ii,jj]
            end
        end
        coeffs[:,l] .= s  # fill column l with value s
    end

    return nothing
end


felang! (generic function with 1 method)

In [5]:
function fouran!(
    gij::Matrix{Float64},
    gil::Matrix{Float64},
    cs::Matrix{Float64},
    m00::Int,
    l00::Int,
    lmin::Vector{Int},
    lmax::Vector{Int},
    mth::Int
)

    # -------------------------------------------------------------------------
    # Purpose:
    #   This routine performs a truncated Fourier transform of gij onto gil
    #   using Fourier coefficients stored in cs.
    #
    # Inputs:
    #   gij(i,j)   : input matrix of size (mth × mth), the "physical-space" data
    #   cs(j,l)    : Fourier coefficient matrix (mth × jmax1)
    #   m00, l00   : integer offsets in the gil matrix
    #   lmin, lmax : define the active range of Fourier mode indices
    #   mth        : number of θ-grid points (dimension of gij along i, j)
    #
    # Output:
    #   gil(i', l') : matrix updated in-place, where i' = m00 + i and l' = l00 + l
    #
    # -------------------------------------------------------------------------

    # Compute jmax1 like Fortran
    jmax1 = lmax[1] - lmin[1] + 1

    # Zero out relevant gil block
    for l1 in 1:jmax1
        for i in 1:mth
            gil[m00 + i, l00 + l1] = 0.0
        end
    end

    # Accumulate with ll offset (critical to match Fortran)
    for l1 in 1:jmax1
        ll = l1 - 1 + lmin[1]
        for j in 1:mth
            for i in 1:mth
                gil[m00 + i, l00 + l1] += cs[j, l1] * gij[i, j]
            end
        end
    end

    return nothing
end

fouran! (generic function with 1 method)

In [6]:
function arrays!(
    mth::Int,
    dth::Float64,
    lmin::Vector{Int},
    lmax::Vector{Int},
    qa1::Float64,
    n::Int,
    delfac::Float64,
    delta::Vector{Float64},
    xpla::Vector{Float64},
    zpla::Vector{Float64},
    grri::Array{Float64,2},
    farwal::Bool
)
    # Sizes
    mth1 = mth + 1
    mth12 = 2 * mth
    jmax1 = lmax[1] - lmin[1] + 1
    q = qa1
    nq = n * q

    # Compute geometry bounds
    xmin = minimum(xpla)
    xmax = maximum(xpla)
    zmin = minimum(zpla)
    zmax = maximum(zpla)

    plrad = 0.5 * (xmax - xmin)
    xmaj = 0.5 * (xmax + xmin)
    delx = plrad * delfac
    delz = plrad * delfac

    println("  delx, dely = ", delx, " ", delz)

    # Poloidal angle grid
    the = [(i-1) * dth for i in 1:mth1]

    # Wall shape (placeholder: straight line wall matching plasma)
    xwal = copy(xpla[1:mth1])
    zwal = copy(zpla[1:mth1])

    # Derivatives (simple finite difference instead of spline)
    xwalp = [ (xwal[mod1(i+1,mth)] - xwal[i]) / dth for i in 1:mth ]
    zwalp = [ (zwal[mod1(i+1,mth)] - zwal[i]) / dth for i in 1:mth ]
    push!(xwalp, xwalp[1])
    push!(zwalp, zwalp[1])

    # Allocate arrays
    cplar = zeros(Float64, mth1, jmax1)
    cplai = zeros(Float64, mth1, jmax1)
    cwallr = zeros(Float64, mth1, jmax1)
    cwalli = zeros(Float64, mth1, jmax1)
    cnqd = zeros(Float64, mth1)
    snqd = zeros(Float64, mth1)
    sinlt = zeros(Float64, mth1, jmax1)
    coslt = zeros(Float64, mth1, jmax1)
    snlth = zeros(Float64, mth1, jmax1)
    cslth = zeros(Float64, mth1, jmax1)

    # Plasma and wall coefficients
    for l1 in 1:jmax1
        for i in 1:mth
            cplar[i,l1] = grri[i,l1]
            cplai[i,l1] = grri[i,jmax1+l1]
            if !farwal
                cwallr[i,l1] = grri[mth+i,l1]
                cwalli[i,l1] = grri[mth+i,jmax1+l1]
            end
        end
        cplar[mth1,l1] = cplar[1,l1]
        cplai[mth1,l1] = cplai[1,l1]
        if !farwal
            cwallr[mth1,l1] = cwallr[1,l1]
            cwalli[mth1,l1] = cwalli[1,l1]
        end
    end

    # Trigonometric basis arrays
    for is in 1:mth1
        theta = (is-1) * dth
        znqd = nq * delta[is]
        cnqd[is] = cos(znqd)
        snqd[is] = sin(znqd)
        for l1 in 1:jmax1
            ll = lmin[1] - 1 + l1
            elth = ll * theta
            elthnq = ll * theta + znqd
            sinlt[is,l1] = sin(elth)
            coslt[is,l1] = cos(elth)
            snlth[is,l1] = sin(elthnq)
            cslth[is,l1] = cos(elthnq)
        end
    end

    return (
        delx, delz, the, xwal, zwal, xwalp, zwalp,
        cplar, cplai, cwallr, cwalli,
        cnqd, snqd, sinlt, coslt, snlth, cslth
    )
end

arrays! (generic function with 1 method)

In [7]:
const pye = π ;
vac_math_path = joinpath(@__DIR__, "..","src", "Vacuum", "Vacuum_math.jl")
include(vac_math_path)

spl1d2! (generic function with 1 method)

In [8]:
"""
    bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
                           xmin::Float64, xmax::Float64,
                           zmin::Float64, zmax::Float64)

Calculates the minimum and maximum X and Z coordinates within a specified range
of two input vectors. This function modifies the provided `Ref` arguments in-place.

# Arguments
- `x1::Vector{Float64}`: Input vector for X coordinates.
- `z1::Vector{Float64}`: Input vector for Z coordinates.
- `n1::Int`: Starting index for the range (Fortran 1-based index).
- `n2::Int`: Ending index for the range (Fortran 1-based index).
- `xmin::Float64`: Reference to store the calculated minimum X value.
- `xmax::Float64`: Reference to store the calculated maximum X value.
- `zmin::Float64`: Reference to store the calculated minimum Z value.
- `zmax::Float64`: Reference to store the calculated maximum Z value.
"""
function bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
                           xmin::Float64, xmax::Float64,
                           zmin::Float64, zmax::Float64)

    # Check whether n1 is higher than n2
    if n1 > n2
        @warn "Starting index n1 ($n1) is greater than ending index n2 ($n2)."
        return nothing
    end
    if n1 < 1 || n2 > length(x1) || n2 > length(z1)
        error("Indices out of bounds.")
    end

    # 최소/최대 값 계산
    xmin = minimum(view(x1, n1:n2))
    xmax = maximum(view(x1, n1:n2))
    zmin = minimum(view(z1, n1:n2))
    zmax= maximum(view(z1, n1:n2))

end

bounds!

In [9]:
"""
    eqarcw(xin, zin, mw1)

This function performs arc length re-parameterization of a 2D curve. It takes an
input curve defined by `(xin, zin)` coordinates and re-samples it such that
the new points `(xout, zout)` are equally spaced in arc length.

# Arguments
- `xin::AbstractVector{Float64}`: Array of x-coordinates of the input curve.
- `zin::AbstractVector{Float64}`: Array of z-coordinates of the input curve.
- `mw1::Int`: Number of points in the input and output curves.

# Returns
- `xout::Vector{Float64}`: Array of x-coordinates of the arc-length re-parameterized curve.
- `zout::Vector{Float64}`: Array of z-coordinates of the arc-length re-parameterized curve.
- `ell::Vector{Float64}`: Array of cumulative arc lengths for the input curve.
- `thgr::Vector{Float64}`: Array of re-parameterized 'theta' values corresponding to equal arc lengths.
- `thlag::Vector{Float64}`: Array of normalized 'theta' values for the input curve (0 to 1).
"""
function eqarcw!(xin::Vector{Float64}, zin::Vector{Float64}, mw1::Int)
    thlag = zeros(Float64, mw1)
    ell = zeros(Float64, mw1)
    thgr = zeros(Float64, mw1)
    xout = zeros(Float64, mw1)
    zout = zeros(Float64, mw1)

    # Initialize thlag as normalized parameter from 0 to 1
    for iw in 1:mw1
        thlag[iw] = (1.0 / (mw1 - 1)) * (iw - 1)
    end

    # Calculate cumulative arc length
    ell[1] = 1.0e-8 # Small non-zero initial length
    for iw in 2:mw1
        thet = (thlag[iw] + thlag[iw - 1]) / 2.0
        
        # Get derivative of xin with respect to thlag
        lagrange1d!(thlag, xin, mw1, 3, thet, nothing , d_xin_d_thlag)
        lagrange1d!(thlag, zin, mw1, 3, thet, nothing, d_zin_d_thlag)
        
        # Calculate arc length segment
        xtzt = norm([d_xin_d_thlag, d_zin_d_thlag])
        ell[iw] = ell[iw - 1] + xtzt / (mw1 - 1)
    end

    # Re-parameterize based on equal arc lengths
    for i in 1:mw1
        elgr = (ell[mw1] / (mw1 - 1)) * (i - 1)
        # Interpolate thlag based on equal arc lengths
        thgr[i], _ = lag(ell, thlag, mw1, 3, elgr, 0)
    end

    # Get xout and zout using the re-parameterized thgr
    for i in 1:mw1
        ttt = thgr[i]
        # Interpolate xin, zin at ttt
        lagrange1d!(thlag, xin, mw1, 3, ttt, xout[i], nothing)
        lagrange1d(thlag, zin, mw1, 3, ttt, zout[i], nothing)
    end

    return xout, zout, ell, thgr, thlag
end

eqarcw!

In [10]:
"""
    adjustb!(betin, betout_ref, a_, bw_, cw_, dw_, xmaj_, plrad_, ishape_)

Adjusts the `betin` angle based on the `ishape_` and other wall/plasma parameters.
This function takes `betout_ref` as a `Ref` so it can modify the output value in-place.

# Arguments
- `betin::Float64`: Input angle.
- `betout_ref::Ref{Float64}`: Reference to the output angle, which will be modified.
- `a_::Float64`: Wall parameter.
- `bw_::Float64`: Wall parameter (elongation or height).
- `cw_::Float64`: Wall parameter (center or offset).
- `dw_::Float64`: Wall parameter (triangularity).
- `xmaj_::Float64`: Magnetic axis X coordinate.
- `plrad_::Float64`: Plasma radius.
- `ishape_::Int`: Integer indicating the wall shape type.
"""
function adjustb!(betin::Float64, betout_ref::Float64, a_::Float64, bw_::Float64, cw_::Float64, dw_::Float64,
                 xmaj_::Float64, plrad_::Float64, ishape_::Int)

    local r0::Float64 = 0.0 # Initialize to avoid UndefVarError if ishape_ is not 21 or 31
    local r::Float64 = 0.0  # Initialize

    if ishape_ == 31
        r0 = cw_
        r  = a_
    elseif ishape_ == 21 # Use elseif for mutually exclusive conditions
        r0 = xmaj_ + cw_ * plrad_
        r  = plrad_ * (1.0 + a_ - cw_)
    else
        @warn "adjustb!: Unsupported ishape_ value: $ishape_. r0 and r will remain 0.0."
    end

    bet2 = betin
    
    # Check for potential division by zero if tan(bet2) is infinite or if bw_ is zero.
    # atan(Inf) is handled correctly by Julia, but tan(bet2) could be very large
    # if bet2 is close to pi/2 + n*pi. If bw_ is zero, division by zero occurs.
    if bw_ == 0.0
        @warn "adjustb!: Division by zero detected (bw_ is 0.0). Setting betout_ref to NaN."
        betout_ref[] = NaN
        return nothing
    end

    betout_ref[] = abs(atan(tan(bet2) / bw_))

    return nothing # Fortran's `return` from a subroutine
end

adjustb!

In [11]:
# Constants and initial values (replace with actual values from your setup)
const nths = 100 # Example value
const pye = π
global aw = 5.0 # Will be saved and restored
global bw = 0.0 # Will be saved and restored
global a = 3.0
global b = 2.0
global cw = 1.0
global dw = 0.0
global tw = 0.0
global abulg = 0.0
global bbulg = 0.0
global tbulg = 0.0
global ishape = 0
global farwal = false
global leqarcw = 0

# Placeholder for arrays that would be in vglobal_mod
global xinf = zeros(Float64, nths) # Assuming size nths based on context
global zinf = zeros(Float64, nths) # Assuming size nths based on context
global xma = 0.0
global zma = 0.0

global xmx = 0.0 # Max x-coordinate, modified and potentially used elsewhere
global zmx = 0.0 # Max z-coordinate, seems to be a typo/unused, zma is used instead
global iplt = 0 # Integer flag, modified to 1 at the end of the function if <= 0

global npots0::Int
global npots::Int
global lfix::Bool
global insect::Bool
global csmin::Float64
global thetatmp::Vector{Float64}
global xwaltmp::Vector{Float64}
global zwaltmp::Vector{Float64}
global xpptmp::Vector{Float64}
global ww1tmp::Vector{Float64}
global ww2tmp::Vector{Float64}
global ww3tmp::Vector{Float64}
global tabtmp::Vector{Float64}
global rioptmp::Vector{Int}

# Variables for common calculations
global awsave::Float64
global bwsave::Float64
global isph::Int
global xmnp::Float64 = 0.0 # Use Ref for single value output from `bounds!`
global xmxp::Float64 = 0.0
global zmnp::Float64 = 0.0
global zmxp::Float64 = 0.0
global xmin::Float64 = 0.0
global xmax::Float64 = 0.0
global zmin::Float64 = 0.0
global zmax::Float64 = 0.0
global plrad::Float64
global xmaj::Float64
global zmid::Float64
global zrad::Float64
global scale::Float64
global delta1::Float64
global mthalf::Int
global inside::Int = 0 # Initialize `inside` as it's incremented



"""
    wwall!(nqnqnq::Int, xwal1::Vector{Float64}, zwal1::Vector{Float64})

This subroutine calculates the coordinates of the wall (xwal1, zwal1) based on
various geometric shapes and plasma parameters. It modifies `xwal1` and `zwal1` in place.

# Arguments
- `nqnqnq::Int`: An integer parameter (its specific use isn't clear from the snippet,
                  but it's passed as an argument).
- `xwal1::Vector{Float64}`: Array to store the calculated x-coordinates of the wall.
                            Modified in-place.
- `zwal1::Vector{Float64}`: Array to store the calculated z-coordinates of the wall.
                            Modified in-place.

c-----------------------------------------------------------------------
c     ishape< 0 Spherical topology
c     ishape<10 Closed toroidal topology
c     ishape<20 Solid conductors not linking plasma
c     ishape<30 Toroidal conductor with a toroidally symmetric gap.
c               Geometry correlated to plasma position and geometry.
c     ishape<40 Toroidal conductor with a toroidally symmetric gap.
c               Geometry independent of plasma.
c-----------------------------------------------------------------------

"""
function wwall!(nqnqnq::Int, xwal1::Vector{Float64}, zwal1::Vector{Float64})

    global aw, bw, a, b, cw, dw, tw, abulg, bbulg, tbulg, ishape, farwal, leqarcw
    global xinf, zinf, xma, zma, xmx, zmx, iplt
    global npots0, npots, lfix, insect, csmin, thetatmp, xwaltmp, zwaltmp, xpptmp, ww1tmp, ww2tmp, ww3tmp, tabtmp, rioptmp
    global awsave, bwsave, isph, xmnp, xmxp, zmnp, zmxp, xmin, xmax, zmin, zmax, plrad, xmaj, zmid, zrad, scale, delta1, mthalf, inside
    global xpp_out, zpp_out # Declare for eqarcw! if they are globals

    iop = Vector{Int}(undef, 2) 
    xpp = zeros(Float64, nths)
    zpp = zeros(Float64, nths)
    ww1 = zeros(Float64, nths)
    ww2 = zeros(Float64, nths)
    ww3 = zeros(Float64, nths)
    thet = zeros(Float64, nths)
    tabx = zeros(Float64, 3)
    tabz = zeros(Float64, 3)

    # Fortran DATA statement
    iplt = 0


    # global aw = 5.0 # Will be saved and restored
    # global bw = 0.0 # Will be saved and restored
    # global a = 3.0
    # global b = 2.0
    # global cw = 1.0
    # global dw = 0.0
    # global tw = 0.0
    # global abulg = 0.0
    # global bbulg = 0.0
    # global tbulg = 0.0
    # global ishape = 0
    # global farwal = false
    # global leqarcw = 0
    
    # # Placeholder for arrays that would be in vglobal_mod
    # global xinf = zeros(Float64, nths) # Assuming size nths based on context
    # global zinf = zeros(Float64, nths) # Assuming size nths based on context
    # global xma = 0.0
    # global zma = 0.0
    
    # global xmx = 0.0 # Max x-coordinate, modified and potentially used elsewhere
    # global zmx = 0.0 # Max z-coordinate, seems to be a typo/unused, zma is used instead
    # global iplt = 0 # Integer flag, modified to 1 at the end of the function if <= 0
    
    # global npots0::Int
    # global npots::Int
    # global lfix::Bool
    # global insect::Bool
    # global csmin::Float64
    # global thetatmp::Vector{Float64}
    # global xwaltmp::Vector{Float64}
    # global zwaltmp::Vector{Float64}
    # global xpptmp::Vector{Float64}
    # global ww1tmp::Vector{Float64}
    # global ww2tmp::Vector{Float64}
    # global ww3tmp::Vector{Float64}
    # global tabtmp::Vector{Float64}
    # global rioptmp::Vector{Int}
    
    # # Variables for common calculations
    # global awsave::Float64
    # global bwsave::Float64
    # global isph::Int
    # global xmnp::Float64 = 0.0 # Use Ref for single value output from `bounds!`
    # global xmxp::Float64 = 0.0
    # global zmnp::Float64 = 0.0
    # global zmxp::Float64 = 0.0
    # global xmin::Float64
    # global xmax::Float64 = 0.0
    # global zmin::Float64
    # global zmax::Float64
    # global plrad::Float64
    # global xmaj::Float64
    # global zmid::Float64
    # global zrad::Float64
    # global scale::Float64
    # global delta1::Float64
    # global mthalf::Int
    # global inside::Int = 0 # Initialize `inside` as it's incremented



    # local aw, bw, ishape, xmx, zma, iplt,xmin, xmax, zmin, zmax


    # Constants calculated based on nths
    mth = nths # Assuming nths is equivalent to mth for indexing purposes in this context
    mth1 = nths # mth1 = mth + 1 in Fortran
    mth2 = nths # mth2 = mth + 2 in Fortran
    dth = 2.0 * pye / (mth2 - 1) # (2.0*pi / (mth+1))

    # --- Computations ---
    xpp[1] = 1.0
    zpp[1] = 1.0
    ww3[1] = 1.0
    tabx[1] = 1.0
    tabz[1] = 1.0

    awsave = aw
    bwsave = bw
    insect = false
    isph = 0

    # --- Shape Logic ---
    if a < -100.0
        isph = 1
        ishape = -10
        bounds!(xinf, zinf, 1, mth, xmnp, xmxp, zmnp, zmxp)
        xmin = xmnp
        xmax = xmxp
        zmin = zmnp
        zmax = zmxp
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        zmid = 0.5 * (zmax + zmin)
        hrad = xmax + aw * (xmax - xmaj)
        vrad = zmax + bw * (zmax - zmid)
        for i in 1:mth1
            xi = xinf[i] - xmaj
            zeta = zinf[i] - zmid
            bbb = (xi * vrad)^2 + (zeta * hrad)^2
            ccc = -xmaj * vrad * xi + hrad * sqrt(bbb - (zeta * xmaj)^2)
            xwal1[i] = xmaj + xi * vrad * ccc / bbb
            zwal1[i] = zmid + zeta * vrad * ccc / bbb
        end
        @goto cleanup # Direct jump to cleanup section
    elseif a > -10.0
        lfix = true
    else
        lfix = false # Initialize lfix for all paths
    end

    if farwal
        @info "No wall"
        return
    end

    xshift = a
    mthalf = floor(Int, mth2 / 2) 
    bounds!(xinf, zinf, 1, mth, xmin, xmax, zmin, zmax)
    plrad = 0.5 * (xmax - xmin)
    xmaj = 0.5 * (xmax + xmin)
    zmid = 0.5 * (zmax + zmin)
    zrad = 0.5 * (zmax - zmin)
    scale = (zmax - zmin)

    if ((xmax - xmin) / 2.0) > scale
        scale = (xmax - xmin) / 2.0
    end

    scale = 1.0
    aw = aw * scale
    bw = bw * scale
    delta1 = dw * (xinf[1] - xma)

    # ishape=2 Elliptical shell
    if ishape == 2
        zh = sqrt(abs(zrad^2 - plrad^2))
        zah = a / zh
        zph = plrad / zh
        zmup = 0.5 * log((zrad + plrad) / (zrad - plrad)) 
        zmuw = log(zah + sqrt(zah^2 + 1)) 
        zxmup = exp(zmup)
        zxmuw = exp(zmuw)
        zbwal = zh * cosh(zmuw) # Major radius of wall
        bw = zbwal / a          # Elongation of wall
        for i in 1:mth2
            the = (i - 1) * dth
            xwal1[i] = xmaj + a * cos(the)
            zwal1[i] = -bw * a * sin(the)
        end
    end

    # ishape=3
    if ishape == 3
        for i in 1:mth2
            rr = (xinf[i] - xma)^2 + (zinf[i] - zma)^2
            ro = sqrt(rr)
            the = atan((zinf[i] - zma), (xinf[i] - xma)) # Use atan2 for correct quadrant
            thex = abs(the)
            lsgn = 1
            if xma > xinf[i]
                the = the + pye
            end
            if i <= mthalf
                if xma > xinf[i]
                    thex = pye - thex
                end
                thet[i] = abs(thex)
            end
            if !lfix
                ro = ro + delta1
                xwal1[i] = xma + lsgn * ro * cos(the)
                zwal1[i] = zma + lsgn * ro * sin(the)
            else
                xshift = (xmax + xmin) / 2.0
                xshift = a
                the = (i - 1) * dth
                xwal1[i] = xshift + aw * cos(the + dw * sin(the))
                zwal1[i] = zma - bw * sin(the)
            end
            if i > mthalf
                continue
            end
            if zwal1[i] < zmin
                continue
            end
            j = i
            insect = false
            jsmall = j
            jlarge = j
            ics = 1
            if xma >= xinf[i]
                ics = -1
            end
            while true
                if zinf[j] >= zwal1[i]
                    jsmall = j
                else
                    break
                end
                if j >= mthalf
                    continue # Fortran's cycle
                end
                if j < 1
                    continue # Fortran's cycle
                end
                j = j + ics
            end
            jlarge = j
            if abs(xinf[jsmall] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if abs(xinf[jlarge] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if !insect
                continue
            end
            inside += 1
        end
    end

    # ishape=4 Modified dee-shaped wall independent of plasma geometry
    if ishape == 4
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = cw + a * cos(the + dw * sin(the))
            zwal1[i] = -bw * a * sin(the + tw * sn2th) - aw * sn2th
        end
    end

    # ishape=5 Dee-shaped wall scaled to plasma
    if ishape == 5
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = xmaj + cw * plrad + plrad * (1.0 + a - cw) * cos(the + dw * sin(the))
            zwal1[i] = -bw * plrad * (1.0 + a - cw) * sin(the + tw * sn2th) - aw * plrad * sn2th
        end
    end

    # ishape=6 Conforming shell
    if ishape == 6
        wcentr = xmaj
        # Fortran's minval(xinf(2:mth1))
        # Note: If xinf is guaranteed to have at least 2 elements, this is safe.
        csmin = min(0.1, 1e-1 * minimum(view(xinf, 2:mth1)))
        for i in 2:mth1
            alph = atan(xinf[i+1] - xinf[i-1], zinf[i-1] - zinf[i+1]) # Fortran's ATAN2
            xwal1[i] = xinf[i] + a * plrad * cos(alph)
            zwal1[i] = zinf[i] + a * plrad * sin(alph)
            # if the wall crosses the R=0 axis, force a thin center stack
            if xwal1[i] <= csmin
                xwal1[i] = csmin
            end
        end
        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]
    end

    # ishape=7 Enclosing bean-shaped wall
    if ishape == 7
        cwr = cw * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = aw * (1.0 + bw * cos(the))
            the2 = cwr * sin(the)
            xofsw = xmax + a * plrad - aw * (1.0 + bw)
            xwal1[i] = xofsw + rho * cos(the2)
            zwal1[i] = -b * rho * sin(the2)
        end
    end

    # ishape=8 Wall of DIII-D
    if ishape == 8
        d3dwall!(xwal1, zwal1, mth, devnull, stdout, 1.0, 0.0)
    end

    # ishape=11 Dee-shaped conductor
    if ishape == 11
        for i in 1:mth2
            the = (i - 1) * dth
            plrad = 0.5 * (xmax - xmin)
            xwal1[i] = xmax + plrad * (a + aw - aw * cos(the + dw * sin(the)))
            zwal1[i] = -plrad * bw * sin(the)
        end
    end

    # ishape=12 Solid bean-shaped conductor on right
    if ishape == 12
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 - aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=13 Solid bean-shaped conductor on left
    if ishape == 13
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 + aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=21 Shell scaled to plasma. Gap on the inner side.
    if ishape == 21
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        a0b = (a0 + plrad * aw) * bw
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = xmaj + cw * plrad

        # Create Ref for adjustb! output
        blgrado_ref = 0.0
        blgradi_ref = 0.0

        # Call adjustb!
        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        dthb = (2.0 * aw * plrad / a0b) * (1.0 - sin(blgrado)) / cos(blgrado)
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2 * thbulg)
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 - aw * plrad * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            fermb = 1.0 / (exp((cost2b - cost2) / tbulg) + 1.0)
            bulge = abulg * plrad * fermb
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=24 Shell scaled to plasma. Gap on the outer side.
    if ishape == 24
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 + aw * plrad * ferm
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=31 Shell independent of plasma. Gap on the inner side.
    if ishape == 31
        a0 = a + aw
        a0b = (a0 + aw) * bw
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = cw

        blgrado_ref = 0.0
        blgradi_ref = 0.0

        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        dthb = (2.0 * aw / a0b) * (1.0 - sin(blgrado)) / cos(blgrado)
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2.0 * thbulg)
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 - aw * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            fermb = 1.0 / (exp((cost2b - cost2) / tbulg) + 1.0)
            bulge = abulg * fermb
            xwal1[i] = cw + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=34 Shell independent of plasma. Gap on the outer side.
    if ishape == 34
        a0 = a + aw
        brad = b * pye / 180.0
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 + aw * ferm
            the2 = brad * sin(the)
            xwal1[i] = cw - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=41 Arbitrary wall generated by spline data from wall_geo.io
    if ishape == 41

        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io) # Skip the next line

            # Allocate arrays based on npots0, as per Fortran's dynamic allocation
            thetatmp = zeros(Float64, npots0)
            xwaltmp = zeros(Float64, npots0)
            zwaltmp = zeros(Float64, npots0)
            rioptmp = zeros(Int, 2)
            xpptmp = zeros(Float64, npots0)
            ww1tmp = zeros(Float64, npots0)
            ww2tmp = zeros(Float64, npots0)
            ww3tmp = zeros(Float64, npots0)
            tabtmp = zeros(Float64, 3)

            for i in 1:npots0
                line = split(readline(io))
                thetatmp[i] = parse(Float64, line[1])
                xwaltmp[i] = parse(Float64, line[2]) - wcentr
                zwaltmp[i] = parse(Float64, line[3])
            end
        end # `do io` block automatically closes the file

        rioptmp[1] = 4
        rioptmp[2] = 4
        spl1d1!(npots0, thetatmp, xwaltmp, xpptmp, rioptmp, 1, ww1tmp, ww2tmp, ww3tmp)

        for i in 1:mth1
            the0 = (i - 1) * dth
            spl1d2(npots0,thetatmp,xwaltmp,xpptmp,1,the0,tabtmp)
            xwal1[i] = tabtmp[1]*a + wcentr
        end

        rioptmp[1] = 4
        rioptmp[2] = 4
        spl1d1!(npots0, thetatmp, xwaltmp, zpptmp, rioptmp, 1, ww1tmp, ww2tmp, ww3tmp)

        for i in 1:mth1
            the0 = (i - 1) * dth
            spl1d2(npots0,thetatmp,zwaltmp,xpptmp,1,the0,tabtmp)
            zwal1[i] =tabtmp[1]*a
        end

        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]
    end

    # ishape=42 Arbitrary wall generated by position data
    if ishape == 42
        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io) # Skip the next line

            if npots0 != mth + 2
                @error "ERROR: Number of points in wall_geo.in must be equal to mth+2."
                error("Wall geometry error") # Stop execution
            end

            thetatmp_dummy = zeros(Float64, npots0)
            for i in 1:npots0
                line = split(readline(io))
                thetatmp_dummy[i] = parse(Float64, line[1]) # Read but not used
                xwal1[i] = parse(Float64, line[2])
                zwal1[i] = parse(Float64, line[3])
            end
        end

        if xwal1[mth1] != xwal1[1] || zwal1[mth1] != zwal1[1]
            @error "ERROR: First point in wall_geo.in must be equal to 2nd last point."
            error("Wall geometry error")
        end
        if xwal1[mth2] != xwal1[2] || zwal1[mth2] != zwal1[2]
            @error "ERROR: Last point in wall_geo.in must be equal to 2nd point."
            error("Wall geometry error")
        end
    end

    xmx = xma + xshift

    # --- Cleanup ---
    @label cleanup # Target for the `goto` from ishape=-10
    if leqarcw == 1
        eqarcw!( xwal1,zwal1, xpp,zpp, ww1,ww2,ww3, mth1 )
        for i in 1:mth1
            xwal1[i] = xpp_out[i]
            zwal1[i] = zpp_out[i]
        end
    end

    if iplt <= 0
        xmx = xmaj
        zma = 0.0
        iplt = 1
    end


    aw = awsave # restore value of aw
    bw = bwsave # restore value of bw

end

wwall!

In [12]:
global n = 1
# Gaussian quadrature weights and points for 8-point integration
const WGAUS = [0.101228536290376, 0.222381034453374, 0.313706645877887, 0.362683783378362,
               0.362683783378362, 0.313706645877887, 0.222381034453374, 0.101228536290376]

const XGAUS = [-0.960289856497536, -0.796666477413627, -0.525532409916329, -0.183434642495650,
                0.183434642495650,  0.525532409916329,  0.796666477413627,  0.960289856497536]

                
ww1 = Vector{Float64}()
ww2 = Vector{Float64}()

"""
    kernel(xobs, zobs, xsce, zsce, j1, j2, isgn, iopw, iops, ischk, params)

Compute kernels of integral equation for Laplace's equation for a torus.

# Arguments
- `xobs`: Observer x coordinates
- `zobs`: Observer z coordinates  
- `xsce`: Source x coordinates
- `zsce`: Source z coordinates
- `j1, j2`: Boundary condition indices
- `isgn`: Sign parameter
- `iopw`: Wall option (0=inactive, 1=active)
- `iops`: Log singularity correction option
- `wall_flag`: Check option for conductor position (ischk)
- `params`: Dictionary containing simulation parameters

# Returns
- `grdgre`: Gradient Green's function matrix
- `gren`: Green's function matrix
"""
function kernel!(X, Z, Xp, Zp, j1, j2, isgn, iopw, iops, wall_flag)

    # matrix output gren is accumulated in grwp of vaccal.
    # While grwp is 𝒢 befor fourier transform, grri is fourier transformed 𝒢

    the = theta_values = LinRange(0, 2*pi, 100)
    thetas = the

    # 1. definition for solving parameters
    wsimpb1=1*dth/3
    wsimpb2=2*dth/3
    wsimpb4=4*dth/3

    algdth = log(dth) # log of dth
    alg = log(2*dth)

    slog1m = 3*dth*(algdth-1/3)
    slog1p = slog1m

    alg0=16.0*dth*(alg-68.0/15.0)/15.0
    alg1=128.0*dth*(alg-8.0/15.0)/45.0
    alg2=4.0*dth*(7.0*alg-11.0/15.0)/45.0

    # 2. check singular points for conductor.
    if wall_flag == true
        # 2.0 initialize jbot and jtop
        jbot=mth/2+1
        jtop=mth/2+1
        
        # 2.1 call wall and restore points of wall in ww1, ww2
        wwall!(mth1,ww1,ww2)

        # 2.2 find where sign of wall r point acrosses zero. 
        # isph means there is 0-corssing point 
        for i in 1:mth
            if ww1[i] * ww1[i+1] ≤ zero
                jbot = ww1[i] > zero ? i : jbot
                jtop = ww1[i] < zero ? i + 1 : jtop
                isph = 1 
            end
        end
    end

    # 3 do spline and calc derivative for Z'_θ and X'_θ in eq.(51)
    xpr = [spline1d_deriv(the, xsce, θ) for θ in thetas]
    zpr = [spline1d_deriv(the, zsce, θ) for θ in thetas]

    # 4, begin obs loop.
    for j in 1:mth 

        # 4,1 initialize variable
        xs=xobs[j] #observation point  # Fixed: () to []
        zs=zobs[j]  # Fixed: () to []
        thes=the[j] # theta value  # Fixed: () to []
        work = zeros(mth1)
        aval1=zero # ∇𝒢_0

        # 4.2 if the point of observation point is in negative, We cannot use green func
        # This is same for source point 
        if xs < zero 
            if j2 == 2 
                work[j] = 1.0  # Fixed: () to []
            end
        else

            # 4.3 set istart and iend
            iend = 2 # end point of integration
            # if wall crossing zero and wall is source, iend set.
            if isph == 1 && j2 == 2
                if jbot - j == 1
                    iend = 3
                elseif jbot - j == 0
                    iend = 4
                elseif j - jtop == 0
                    iend = 0
                elseif j - jtop == 1
                    iend = 1
                end
            end
            istart = 4 - iend # starting point of integration

            # 4-3 on mth. then mths is equals to mtheta+3 ??? I'm not sure
            mths=mth-(istart+iend-1) 

            
            # 4.4 loop for source point 
            for i in 1:mths

                # 4.5 get source point index(ic) theta(theta), X(xt), and Z(zt)
                ic = i + j + istart - 1
                if ic ≥ mth1
                    ic = ic - mth
                end
                theta=(ic-1)*dth 
                xt=xsce[ic]  # Fixed: () to []
                zt=zsce[ic]  # Fixed: () to []

                # 4.6 if source point is in negative, we cannot use green function
                # & if source point(ic) and obs point (j) is same, it's singular
                if xt < 0 
                    continue 
                end
                if ic == j 
                    continue 
                end

                # 4.7 calc X'_θ (xtp) and Z'_θ (ztp) and call green function
                # aval is 𝒥 ∇'𝒢ⁿ∇'ℒ, bval is 2pi𝒢ⁿ. aval0 is 𝒥 ∇'𝒢ⁿ∇'ℒ for n=0
                xtp=xpr[ic]  # Fixed: () to []
                ztp=zpr[ic]  # Fixed: () to []
                G, aval, aval0, bval = green(xs,zs,xt,zt,xtp,ztp,n,usechancebugs=false)

                # 4.8 simpson integral. 4 for odd, 2 for even, and 1 for others.
                wsimpb=wsimpb2
                if (i÷2)*2 == i  # Fixed: / to ÷ for integer division
                    wsimpb=wsimpb4
                end
                if (i == 1)||(i == mths)
                    wsimpb=wsimpb1
                end
                wsimpa=wsimpa2
                if (i÷2)*2 == i  # Fixed: / to ÷ for integer division
                    wsimpa=wsimpa4
                end
                if (i == 1)||(i == mths)
                    wsimpa=wsimpa1
                end

                # 4.9 work and gren
                # work : simpson integral for aval(𝒥 ∇'𝒢ⁿ∇'ℒ)
                # gren : log singularity values accumulated. simpson integral for bval
                # aval1 : aval1
                work[ic]=work[ic]+isgn*aval*wsimpa  # Fixed: () to []
                gren[j,ic]=gren[j,ic]+bval*wsimpb # integral of bval (log singularity)  # Fixed: () to []
                aval1 = aval1 + aval0 * wsimpa
            
            end
            
            # 5.1 if it's plasma/plsama, wall/wall and in negative wall point, skip loop
            # obs : j1 = 1(plasma), wall(2)
            # src : j1 = 1(plasma), wall(2)
            j1j2 = j1+j2
            if j1j2 != 2 && isph == 1 && j > jbot && j < jtop  # Fixed: a&& to &&
                continue
            end

            # 5.2 Get js
            # js2 : j - iend + 1
            thes = the[j]  # Fixed: () to []
            js1=mod(j-iend+mth-1,mth)+1
            js2=mod(j-iend+mth,mth)+1
            js3=mod(j-iend+mth+1,mth)+1
            js4=mod(j-iend+mth+2,mth)+1
            js5=mod(j-iend+mth+3,mth)+1

            # 6 Singular points when source point and obs point are the same
            # 6.1 integration each left and right
            for ilr in [1,2]
                xl = thes + (2*ilr-iend-2)*dth
                xu = xl + 2 * dth
                agaus = (xu + xl)/2
                bgaus = (xu - xl)/2
                tgaus = agaus .+ XGAUS .* bgaus # tgaus is 8 point gauss points, since xgauss is for only [-1,1]  # Fixed: xgaus to XGAUS
                # 6.2 for each 8 gaussian points
                for ig in 1:8
                    tgaus0 = tgaus[ig] #i-th value of for 8 points, in theta  # Fixed: () to []
                    tgaus0 = mod(tgaus0, twopi)

                    # 6.3 get X, X', Z, Z' for gaussian point
                    spl1d2!(mth1,the,xsce,xpp,1,tgaus0,tab)
                    xt = tab[1] # xt is X  # Fixed: () to []
                    xtp = tab[2] # xtp is X'_θ  # Fixed: () to []
                    spl1d2!(mth1,the,zsce,zpp,1,tgaus0,tab)
                    zt=tab[1] # zt is Z  # Fixed: () to []
                    ztp=tab[2] # ztp is Z'_θ  # Fixed: () to []

                    # 6.4 call green function
                    G, aval, aval0, bval = green(xs,zs,xt,zt,xtp,ztp,n,usechancebugs=false)

                    # 6.5 add logarithm on G (not 𝒢_n). Chance eq.(75)
                    # iops = 1
                    bval = G + iops * log((thes-tgaus[ig])^2)/xs  # Fixed: () to []

                    # 6.6 calc wgaus. bgaus refers Δ in theta. wgaus is weight for each 8 points
                    wgbg=WGAUS[ig]*bgaus  # Fixed: wgaus() to WGAUS[]

                    # 6.7 calc pgaus. below Chance eq.(77)
                    pgaus=(tgaus[ig]-thes-(2-iend)*dth)/dth  # Fixed: () to []
                    pgaus2=pgaus*pgaus
                    amm = (pgaus2-1)*pgaus*(pgaus-2)/24.0 *wgbg
                    am = -(pgaus-1)*pgaus*(pgaus2-4)/6.0 *wgbg
                    a0 = (pgaus2-1)*(pgaus2-4)/4.0 *wgbg
                    ap = -(pgaus+1)*pgaus*(pgaus2-4)/6.0 *wgbg
                    app = (pgaus2-1)*pgaus*(pgaus+2)/24.0 *wgbg

                    # 6.8 add up in work
                    work[js1] += isgn * aval * amm  # Fixed: () to []
                    work[js2] += isgn * aval * am  # Fixed: () to []
                    work[js3] += isgn * aval * a0  # Fixed: () to []
                    work[js4] += isgn * aval * ap  # Fixed: () to []
                    work[js5] += isgn * aval * app  # Fixed: () to []

                    # 6.9 minus diverging value
                    work[j] -= isgn * aval0 * wgbg  # Fixed: () to []
                    if j == jres
                        ak0i -= isgn * aval0 * wgbg
                    end
                    
                    # 6.10 skip when plasma, no skip when considering wall
                    if iopw == 0  # Fixed: opw to iopw
                        continue
                    end

                    # 6.11 if Wall is considered(wall/wall, plasma/wall, wall/plasma), add up bval value
                    gren[j,js1] += bval * amm  # Fixed: () to []
                    gren[j,js2] += bval * am  # Fixed: () to []
                    gren[j,js3] += bval * a0  # Fixed: () to []
                    gren[j,js4] += bval * ap  # Fixed: () to []
                    gren[j,js5] += bval * app  # Fixed: () to []

                end
            end


            # 7. Residue
            # 7.1 Set default residu
            if j1 == j2 
                residu = 2.0
            else
                residu = 0.0
            end

            # 7.2 Change resdg, resk0 according to ishape
            # resdg, resk0 ???
            if ishape < 10 || ishape == 41 || ishape == 42
                resdg=(2-j1)*(2-j2)+(j1-1)*(j2-1)
                resk0=(2-j1)*(2-j2)+(j1-3)*(j2-1)
                residu=resdg+resk0
            end

            # 7.3 minus residu value
            work[j] -= (isgn * aval1 + residu)  # Fixed: () to []
            if j == jres
                ak0i -= isgn * aval1
            end


            # 8.1 Only when plasma/plasma, log singularity activate. (S1)
            if iops == 1 && iopw != 0
                gren[j,js1] -= alg2 / xs  # Fixed: () to []
                gren[j,js2] -= alg1 / xs  # Fixed: () to []
                gren[j,js3] -= alg0 / xs  # Fixed: () to []
                gren[j,js4] -= alg1/xs  # Fixed: () to []
                gren[j,js5] -= alg2/xs  # Fixed: () to []
            end

        end

        # 4.3 Store all the datas of work in grdgre, gren
        grdgre[(j1-1)*mth + j, (j2-1)*mth + (1:mth)] .= work[1:mth]
        gren[j, 1:mth] ./= twopi

    end
    
    return grdgre, gren
end

kernel!

In [13]:
# Global dictionary to store named timestamps
const _msc_timer_dict = Dict{Symbol,Float64}()

function msctimer(outmod::IO, checkpoint_name::AbstractString)
    t = time()  # current time in seconds since epoch (Float64)
    if haskey(_msc_timer_dict, :last)
        dt = t - _msc_timer_dict[:last]
        println(outmod, "Timer checkpoint '$checkpoint_name': +$(round(dt, digits=4)) seconds since last checkpoint.")
    else
        println(outmod, "Timer checkpoint '$checkpoint_name': starting at $(t).")
    end
    _msc_timer_dict[:last] = t
    flush(outmod)
end


msctimer (generic function with 1 method)

In [14]:
function dbg(label::String, arr; precision=6, maxprint=10)
    if DEBUG_VACCAL
        println("=== $label ===")
        flat = vec(arr)
        n = min(length(flat), maxprint)
        for i in 1:n
            println("  [$i] = ", round(flat[i], digits=precision))
        end
        if length(flat) > n
            println("  ... (", length(flat) - n, " more entries)")
        end
        println()
    end
end

function dbg_scalar(label::String, val; precision=6)
    if DEBUG_VACCAL
        println(">>> $label = ", round(val, digits=precision))
    end
end

dbg_scalar (generic function with 1 method)

In [15]:
function vaccal!(
    # numerical parameters
    n, q, qa1, nq, mth, mth1, mth2, mthin1, mfel,
    lmin, lmax, lfele, lfour, lgato, ladj, ldcon, lgpec, lnova,
    lsymz, lspark, lgivup, lsym, lpest1, lcheck1, check1, check2, checkd, checke,
    farwal, ishape, kernelsign, jtop, jbot,
    twopi, twopi2, dth,

    # geometry arrays
    xpla, zpla, xwal, zwal, xplap, zplap, xwalp, zwalp,
    cnqd, snqd, cslth, snlth, cn0,
    xjacob, xjdtxj,

    # workspace sizes
    nfm, nfmsq, nfm2, nths, nths2, mtot,

    # output arrays
    vacmat, vacmti, vacmatu, vacmtiu, ajll, chiwc, chiws, xpass, zpass, gatovac, 
    delta, ff,

    # I/O
    outmod, iotty, iodsk, iovac, jobid
)

    # ----------------------------------------------------------
    # Allocate and zero arrays
    # ----------------------------------------------------------
    grdgre = zeros(Float64, nths2, nths2)
    grwp   = zeros(Float64, nths2, nths2)
    dummy  = zeros(Float64, nths2, nfm2)
    ajll  .= 0.0
    vacmat .= 0.0
    vacmti .= 0.0

    println(outmod, "\n          Matrix Storage: K(obs_ji,sou_ji):\n")
    println(outmod, "          j = observer points, i = source points.")
    println(outmod, "          ie. K operates on chi from the left.\n")
    println(outmod, "          Observer Source  Block")
    println(outmod, "          plasma  plasma   1  1")
    println(outmod, "          plasma  wall     1  2")
    println(outmod, "          wall    plasma   2  1")
    println(outmod, "          wall    wall     2  2\n")

    # Initialization
    xwal[1] = 0.0
    zwal[1] = 0.0
    factpi  = twopi
    jmax    = 2*lmax[1] + 1
    jmax1   = lmax[1] - lmin[1] + 1
    lmax1   = lmax[1] + 1
    ln      = lmin[1]
    lx      = lmax[1]
    jdel    = 8
    nq      = n*q
    tmth    = 2*mth
    mthsq   = tmth * tmth
    lmth    = tmth * 2*jmax1
    j1v     = nfm
    j2v     = nfm

    if check1
        msctimer(outmod, "before kernels")
    end

    # ----------------------------------------------------------
    # Apply wall boundary conditions
    # ----------------------------------------------------------
    wwall!(mth, xwal, zwal)

    # ----------------------------------------------------------
    # Plasma–Plasma block
    # ----------------------------------------------------------
    j1, j2 = 1, 1
    ksgn = 2*j2 - 3
    #grdgre, grwp = kernel!(xpla, zpla, xpla, zpla, j1, j2, ksgn, 1, 1, 0)

    if lfele == 1
        felang!(grwp, grdgre, cnqd, 0,0)
        felang!(grwp, grdgre, snqd, 0,jmax1)
    end
    if lfour == 1
        fouran!(grwp, grdgre, cslth, 0, 0, lmin, lmax, mth)
        fouran!(grwp, grdgre, snlth, 0, jmax1, lmin, lmax, mth)
    end

    # After computing plasma-plasma kernel
    dbg("grwp (plasma-plasma)", grwp)
    dbg("grdgre (plasma-plasma)", grdgre)

    # ----------------------------------------------------------
    # Plasma–Wall block
    # ----------------------------------------------------------
    j1, j2 = 1, 2
    ksgn = 2*j2 - 3
    grpw_block = similar(grdgre)
    #grdgre, grpw_block = kernel!(xpla, zpla, xwal, zwal, j1, j2, ksgn, 1, 1, 0)

    if lfele == 1
        felang!(grpw_block, grdgre, cnqd, 0,0)
        felang!(grpw_block, grdgre, snqd, 0,jmax1)
    end
    if lfour == 1
        fouran!(grpw_block, grdgre, cslth, 0, 0, lmin, lmax, mth)
        fouran!(grpw_block, grdgre, snlth, 0, jmax1, lmin, lmax, mth)
    end

    # After plasma-wall kernel
    dbg("grpw_block (plasma-wall)", grpw_block)

    # ----------------------------------------------------------
    # Wall–Plasma block
    # ----------------------------------------------------------
    j1, j2 = 2, 1
    ksgn = 2*j2 - 3
    grwpw_block = similar(grdgre)
    #grdgre, grwpw_block = kernel!(xwal, zwal, xpla, zpla, j1, j2, ksgn, 1, 1, 0)

    if lfele == 1
        felang!(grwpw_block, grdgre, cnqd, 0,0)
        felang!(grwpw_block, grdgre, snqd, 0,jmax1)
    end
    if lfour == 1
        fouran!(grwpw_block, grdgre, cslth, 0, 0, lmin, lmax, mth)
        fouran!(grwpw_block, grdgre, snlth, 0, jmax1, lmin, lmax, mth)
    end

    # After wall-plasma kernel
    dbg("grwpw_block (wall-plasma)", grwpw_block)

    # ----------------------------------------------------------
    # Wall–Wall block
    # ----------------------------------------------------------
    j1, j2 = 2, 2
    ksgn = 2*j2 - 3
    grww_block = similar(grdgre)
    #grdgre, grww_block = kernel!(xwal, zwal, xwal, zwal, j1, j2, ksgn, 1, 1, 0)

    if lfele == 1
        felang!(grww_block, grdgre, cnqd, 0,0)
        felang!(grww_block, grdgre, snqd, 0,jmax1)
    end
    if lfour == 1
        fouran!(grww_block, grdgre, cslth, 0, 0, lmin, lmax, mth)
        fouran!(grww_block, grdgre, snlth, 0, jmax1, lmin, lmax, mth)
    end

    # After wall-wall kernel
    dbg("grww_block (wall-wall)", grww_block)

    # ----------------------------------------------------------
    # Assemble matrices for solving
    # ----------------------------------------------------------
    assemble_vacuum_matrix!(
        vacmat, vacmti, vacmatu, vacmtiu,
        grwp, grpw_block, grwpw_block, grww_block,
        mth, jmax1, ln, lx, factpi
    )

    # After assembling vacmat
    dbg("vacmat assembled", vacmat)
    dbg_scalar("factpi", factpi)

    # ----------------------------------------------------------
    # Solve vacuum system
    # ----------------------------------------------------------
    solve_vacuum!(
        vacmat, vacmti, vacmatu, 
        vacmtiu, ajll, chiwc
    )

    # After solving vacuum
    dbg("ajll", ajll)
    dbg("chiwc", chiwc)
    dbg("chiws", chiws)

    println("vacmat size = ", size(vacmat))
    println("ajll size = ", length(ajll))
    println("chiwc size = ", length(chiwc))

    # ----------------------------------------------------------
    # Call arrays! here with proper parameters
    # ----------------------------------------------------------
    grri = zeros(Float64, nths2, nths2)    # allocate grri
    delfac = 0.5                          # example value; set as needed
    arrays!(mth, dth, lmin, lmax, qa1, n, delfac, delta, xpla, zpla, grri, farwal != 0)

    # ----------------------------------------------------------
    # Finish up: timing, writing, cleanup
    # ----------------------------------------------------------
    if check1
        msctimer(outmod, "after vacuum solve")
    end

    return
end

vaccal! (generic function with 1 method)

In [16]:
global DEBUG_VACCAL = true  # Set to false to disable debug output

function test_vaccal()
    # Minimal realistic parameters (adjust as needed)
    n       = 1
    q       = 1
    qa1     = 0.0
    nq      = n * q
    mth     = 2
    mth1    = mth + 1
    mth2    = 2 * mth
    mthin1  = 0  # or set as needed
    mfel    = 0  # or set as needed

    lmin    = [0]
    lmax    = [3]

    lfele   = 1
    lfour   = 1
    lgato   = 0
    ladj    = 0
    ldcon   = 0
    lgpec   = 0
    lnova   = 0
    lsymz   = 0
    lspark  = 0
    lgivup  = 0
    lsym    = 0
    lpest1  = 0
    lcheck1 = true
    check1  = true
    check2  = false
    checkd  = false
    checke  = false

    farwal  = 1
    ishape  = 0
    kernelsign = 1
    jtop    = 0
    jbot    = 0

    twopi   = 2 * π
    twopi2  = 4 * π
    dth     = 2π / 16

    nths    = 16
    nths2   = 32
    nfm     = 4
    nfmsq   = 16
    nfm2    = 8
    mtot    = 1

    # Geometry arrays: initialize with simple dummy values (e.g., linspace or zeros)
    xpla   = collect(LinRange(1.0, 2.0, nths))
    zpla   = collect(LinRange(1.0, 2.0, nths))
    xwal   = zeros(nths)
    zwal   = zeros(nths)
    xplap  = zeros(nths)
    zplap  = zeros(nths)
    xwalp  = zeros(nths)
    zwalp  = zeros(nths)
    cnqd   = zeros(nths, nfm)
    snqd   = zeros(nths, nfm)
    cslth  = zeros(nths, nfm)
    snlth  = zeros(nths, nfm)
    cn0    = zeros(nths, nfm)
    xjacob = zeros(nths)
    xjdtxj = zeros(nths)

    # Output arrays: allocate with correct sizes
    size_vacmat = (2*mth*(lmax[1]-lmin[1]+1), 2*mth*(lmax[1]-lmin[1]+1))
    vacmat  = zeros(Float64, size_vacmat...)
    vacmti  = similar(vacmat)
    vacmatu = similar(vacmat)
    vacmtiu = similar(vacmat)
    ajll    = zeros(Float64, nfm)
    chiwc   = zeros(Float64, nfm)
    chiws   = zeros(Float64, nfm)
    xpass   = zeros(Float64, nths)
    zpass   = zeros(Float64, nths)
    gatovac = zeros(Float64, nfmsq)
    delta   = zeros(Float64, nfm)
    ff      = zeros(Float64, nfm)

    # Use stdout for printing
    outmod = stdout
    iotty  = stdout
    iodsk  = stdout
    iovac  = stdout
    jobid  = "test_job"

    # Run vaccal!
    try
        vaccal!(
            n, q, qa1, nq, mth, mth1, mth2, mthin1, mfel,
            lmin, lmax, lfele, lfour, lgato, ladj, ldcon, lgpec, lnova,
            lsymz, lspark, lgivup, lsym, lpest1, lcheck1, check1, check2, checkd, checke,
            farwal, ishape, kernelsign, jtop, jbot,
            twopi, twopi2, dth,
            xpla, zpla, xwal, zwal, xplap, zplap, xwalp, zwalp,
            cnqd, snqd, cslth, snlth, cn0,
            xjacob, xjdtxj,
            nfm, nfmsq, nfm2, nths, nths2, mtot,
            vacmat, vacmti, vacmatu, vacmtiu, ajll, chiwc, chiws, xpass, zpass, gatovac,
            delta, ff,
            outmod, iotty, iodsk, iovac, jobid
        )
        println("vaccal! executed successfully")
    catch e
        println("vaccal! failed with error:")
        println(e)
        println(stacktrace(e))
    end

    # Inspect some outputs
    println("vacmat[1:4, 1:4] =")
    for row in eachrow(vacmat[1:4, 1:4])
        println(row)
    end
    println("ajll = ", ajll)
    println("chiwc = ", chiwc)
end

# Run the test
test_vaccal()



          Matrix Storage: K(obs_ji,sou_ji):

          j = observer points, i = source points.
          ie. K operates on chi from the left.

          Observer Source  Block
          plasma  plasma   1  1
          plasma  wall     1  2
          wall    plasma   2  1
          wall    wall     2  2

Timer checkpoint 'before kernels': starting at 1.753906036367896e9.
=== grwp (plasma-plasma) ===
  [1] = 0.0
  [2] = 0.0
  [3] = 0.0
  [4] = 0.0
  [5] = 0.0
  [6] = 0.0
  [7] = 0.0
  [8] = 0.0
  [9] = 0.0
  [10] = 0.0
  ... (1014 more entries)

=== grdgre (plasma-plasma) ===
  [1] = 0.0
  [2] = 0.0
  [3] = 0.0
  [4] = 0.0
  [5] = 0.0
  [6] = 0.0
  [7] = 0.0
  [8] = 0.0
  [9] = 0.0
  [10] = 0.0
  ... (1014 more entries)

=== grpw_block (plasma-wall) ===
  [1] = 0.0
  [2] = 0.0
  [3] = 0.0
  [4] = 0.0
  [5] = 0.0
  [6] = 3.481436193625305e227
  [7] = 3.634613562675448e185
  [8] = 0.0
  [9] = 0.0
  [10] = 0.0
  ... (1014 more entries)

=== grwpw_block (wall-plasma) ===
  [1] = 0.0
  [2] =

┌ Warning: Vacuum solve failed, using fallback (zeros)
│   exception =
│    ArgumentError: matrix contains Infs or NaNs
│    Stacktrace:
│      [1] chkfinite
│        @ /Applications/Julia-1.11.app/Contents/Resources/julia/share/julia/stdlib/v1.11/LinearAlgebra/src/lapack.jl:105 [inlined]
│      [2] getrf!(A::Matrix{Float64}, ipiv::Vector{Int64}; check::Bool)
│        @ LinearAlgebra.LAPACK /Applications/Julia-1.11.app/Contents/Resources/julia/share/julia/stdlib/v1.11/LinearAlgebra/src/lapack.jl:582
│      [3] getrf!
│        @ /Applications/Julia-1.11.app/Contents/Resources/julia/share/julia/stdlib/v1.11/LinearAlgebra/src/lapack.jl:580 [inlined]
│      [4] getrf!
│        @ /Applications/Julia-1.11.app/Contents/Resources/julia/share/julia/stdlib/v1.11/LinearAlgebra/src/lapack.jl:787 [inlined]
│      [5] #lu!#182
│        @ /Applications/Julia-1.11.app/Contents/Resources/julia/share/julia/stdlib/v1.11/LinearAlgebra/src/lu.jl:91 [inlined]
│      [6] lu!
│        @ /Applications/Julia-1.

=== ajll ===
  [1] = 0.0
  [2] = 0.0
  [3] = 0.0
  [4] = 0.0

=== chiwc ===
  [1] = 0.0
  [2] = 0.0
  [3] = 0.0
  [4] = 0.0

=== chiws ===
  [1] = 0.0
  [2] = 0.0
  [3] = 0.0
  [4] = 0.0

vacmat size = (16, 16)
ajll size = 4
chiwc size = 4
  delx, dely = 0.25 0.25
Timer checkpoint 'after vacuum solve': +0.8543 seconds since last checkpoint.
vaccal! executed successfully
vacmat[1:4, 1:4] =
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
ajll = [0.0, 0.0, 0.0, 0.0]
chiwc = [0.0, 0.0, 0.0, 0.0]


In [ ]:
function ent33()
    @error "cardmo/inglo/dskmd1 has not finished"
    # cardmo()
    # inglo()
    # dskmd1()
    # funint()
    
    if !wall
        vaccal()
    end

    return
end